In [1]:
import json
import pandas as pd

In [2]:
class Fault(object):
    def __init__(self, id, ip, iter, bit, func, latency):
        self.id = id
        self.ip = ip
        self.iter = iter
        self.bit = bit
        self.func = func
        self.latency = latency
        
    
    
    def __repr__(self):
        return str({'ID': self.id, 'IP': self.ip, 'iter': self.iter, 'bit': self.bit, 'latency': self.latency})
        print("(", self.id, self.ip, self.iter, self.bit, self.latency, ")")
        # print("(ID: %s, IP: %s, Iter: %s, Bit: %d, Latency: %d)" % (self.id, self.ip, self.iter, self.bit, self.latency))

    def __str__(self):
        return str({'ID': self.id, 'IP': self.ip, 'iter': self.iter, 'bit': self.bit, 'latency': self.latency})
    
    def __lt__(self, other):
        return self.id < other.id
        
def json_parser(ffile, filter='SIGSEGV'):
    ID = list()
    IP = list()
    IT = list()
    BIT = list()
    Func = list()
    Latency = list()
    
    with open(ffile) as fh:
        for line in fh.readlines():
            record = json.loads(line)

            status = record['status']
            if status != filter:
                continue
            
            ID.append(record['id'])
            IP.append(record['fault']['dyn_inst'][0])
            IT.append(record['fault']['dyn_inst'][1])
            BIT.append(record['fault']['bit_flipped'])
            Func.append(record['fault']['parent_func'])
            Latency.append(len(record['track']))
    d = {'ID': ID, 'IP': IP, 'Iter': IT, 'Bit': BIT, 'Func': Func, 'Latency': Latency}
    return pd.DataFrame(d)

In [9]:
ffile = '/home/cchen/Documents/Projects/CARE/Benchmarks/miniFE/O0.bak/screen/screen_faults.json'
data = json_parser(ffile)
print(len(data))

5459


In [5]:
ranges = [0, 10, 50, 100, 400, 450]
latency = data['Latency']
dist = latency.groupby(pd.cut(latency, ranges)).count()
print(dist)

Latency
(0, 10]       3226
(10, 50]       812
(50, 100]       71
(100, 400]      51
(400, 450]     450
Name: Latency, dtype: int64


In [7]:
funcs = data['Func']
funcs.value_counts()

miniFE::matvec_std<miniFE::CSRMatrix<double, int, int>, miniFE::Vector<double, int, int> >::operator()                                                                     2153
miniFE::daxpby<miniFE::Vector<double, int, int> >                                                                                                                           370
std::_Rb_tree<int, int, std::_Identity<int>, std::less<int>, std::allocator<int> >::_M_lower_bound                                                                          324
std::__lower_bound<int*, int, __gnu_cxx::__ops::_Iter_less_val>                                                                                                             256
miniFE::Hex8::diffusionMatrix_symm<double>                                                                                                                                  151
miniFE::find_row_for_id<int>                                                                                            